1 단계 랭체인 설정

In [4]:
pip install "langchain>=0.2" langchain-openai langchain-community pydantic pillow googlemaps tavily-python python-dateutil

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------- ----------------------- 0.4/1.0 MB 12.9 MB/s eta 0:00:01
   -------------------------------------- - 1.0/1.0 MB 15.7 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 12.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/74.4 kB ? eta -:--:--
   ---------------------------------------- 74.4/74.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.5 MB 15.9 MB/s eta 0:00:01
   ----------------- ---------------------- 1.1/2.5 MB 14.2 MB/s eta 0:00:01
   --------------------------- ------------ 1.7/2.5 MB 15.8 MB/s eta 0:00:01
   ------------------------------------ --- 2.3/2.5 MB 16.2 MB/s eta 0:00:01
  

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [76]:
# -*- coding: utf-8 -*-
"""
process_image()의 기존 흐름/시그니처를 유지하면서,
LangChain을 써서 '장소 설명'을 넘어 '무슨 이벤트가 언제/왜 일어났는지'까지
추론/검색해 EventCard로 함께 반환하는 드롭인 교체 버전.

검색은 **Tavily만** 사용합니다. (tavily-python + langchain-community)

필요 패키지
pip install "langchain>=0.2" langchain-openai langchain-community pydantic pillow googlemaps tavily-python python-dateutil

환경변수(권장)
- OPENAI_API_KEY : OpenAI 키 (함수 인자 openai_api_key로도 주입 가능)
- TAVILY_API_KEY : Tavily 키(없으면 검색 생략)

기존 반환값을 보존하면서, result["event_card"]를 추가로 돌려줍니다.
"""
from __future__ import annotations
from typing import Optional, List, Dict, Any
from dataclasses import dataclass

import os
import base64
from datetime import datetime, timezone

from PIL import Image, ExifTags
from dateutil import parser as dateparser
from typing import Dict, Any, List, Optional

# LangChain
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.messages import SystemMessage, HumanMessage

try:
    # 검색은 Tavily만!
    from langchain_community.tools.tavily_search import TavilySearchResults
    _HAVE_TAVILY = True
except Exception:  # pragma: no cover
    _HAVE_TAVILY = False

# =====================
# OpenAI/Azure/Legacy API 스위처 (chat.completions.create 통일 인터페이스)
# =====================
from typing import Callable
import openai as _openai_legacy

def get_chat_create(
    api: str = "openai",                # "openai" | "legacy" | "azure"
    api_key: Optional[str] = None,
    base_url: Optional[str] = None,     # Azure endpoint 등
    api_version: Optional[str] = None,  # Azure API 버전 (예: "2024-02-15-preview")
) -> Callable[..., Any]:
    """
    반환: chat.completions.create(**kwargs)처럼 호출 가능한 함수.
    기존 코드의 client.chat.completions.create 자리에 그대로 넣어 쓸 수 있음.
    """
    if api == "openai":
        try:
            from openai import OpenAI
            client = OpenAI(api_key=api_key, base_url=base_url) if base_url else OpenAI(api_key=api_key)
            return client.chat.completions.create
        except Exception:
            _openai_legacy.api_key = api_key
            def _create(**kwargs):
                return _openai_legacy.ChatCompletion.create(**kwargs)
            return _create
    elif api == "legacy":
        _openai_legacy.api_key = api_key
        def _create(**kwargs):
            return _openai_legacy.ChatCompletion.create(**kwargs)
        return _create
    elif api == "azure":
        from openai import AzureOpenAI
        client = AzureOpenAI(api_key=api_key, azure_endpoint=base_url, api_version=api_version or "2024-02-15-preview")
        return client.chat.completions.create
    else:
        from openai import OpenAI
        client = OpenAI(api_key=api_key, base_url=base_url) if base_url else OpenAI(api_key=api_key)
        return client.chat.completions.create

# =====================
# LangChain 기반 이벤트 확장 유틸 (검색은 Tavily만)
# =====================

class VisionClues(BaseModel):
    scene_summary: str
    visible_text: List[str] = []
    notable_objects: List[str] = []
    time_clues: List[str] = []
    place_clues: List[str] = []
    event_guess: Optional[str] = None
    candidate_queries: List[str] = []


class SourceItem(BaseModel):
    url: str
    title: Optional[str] = None


class EventCard(BaseModel):
    method: str
    event_title: Optional[str] = None
    event_type: Optional[str] = None
    occurred_time_utc: Optional[str] = None
    occurred_time_confidence: float = 0.0
    location_name: Optional[str] = None
    latitude: Optional[float] = None
    longitude: Optional[float] = None
    why_summary: Optional[str] = None
    what_happened: Optional[str] = None
    who_involved: Optional[str] = None
    key_evidence: List[str] = []
    sources: List[SourceItem] = []
    notes: Optional[str] = None


def _set_openai_key(openai_api_key: Optional[str]):
    if openai_api_key:
        os.environ["OPENAI_API_KEY"] = openai_api_key


def _b64image(image_path: str) -> str:
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")


2단계 필수 함수들 수록

In [80]:

# =====================
# 기존 헬퍼 (시그니처 유지)
# =====================

def has_gps_metadata(image_path: str) -> bool:
    try:
        img = Image.open(image_path)
        exif_raw = img._getexif() or {}
        if not exif_raw:
            return False
        tagmap = {ExifTags.TAGS.get(k, k): v for k, v in (exif_raw or {}).items()}
        return "GPSInfo" in tagmap
    except Exception:
        return False


def _dms_to_deg(dms, ref):
    def _r(x):
        return float(x[0]) / float(x[1]) if isinstance(x, tuple) else float(x)
    deg = _r(dms[0]) + _r(dms[1]) / 60.0 + _r(dms[2]) / 3600.0
    if ref in ["S", "W"]:
        deg = -deg
    return deg


def extract_gps(image_path: str):
    try:
        img = Image.open(image_path)
        exif_raw = img._getexif() or {}
        tagmap = {ExifTags.TAGS.get(k, k): v for k, v in (exif_raw or {}).items()}
        gps = tagmap.get("GPSInfo")
        if not gps:
            return None, None
        gps = {ExifTags.GPSTAGS.get(k, k): v for k, v in gps.items()}
        lat_dms, lat_ref = gps.get("GPSLatitude"), gps.get("GPSLatitudeRef")
        lon_dms, lon_ref = gps.get("GPSLongitude"), gps.get("GPSLongitudeRef")
        if lat_dms and lat_ref and lon_dms and lon_ref:
            return _dms_to_deg(lat_dms, lat_ref), _dms_to_deg(lon_dms, lon_ref)
    except Exception:
        pass
    return None, None


def _extract_exif_datetime(image_path: str) -> Optional[datetime]:
    try:
        img = Image.open(image_path)
        exif_raw = img._getexif() or {}
        tagmap = {ExifTags.TAGS.get(k, k): v for k, v in (exif_raw or {}).items()}
        for key in ("DateTimeOriginal", "DateTimeDigitized", "DateTime"):
            if key in tagmap:
                try:
                    return dateparser.parse(str(tagmap[key]).replace(":", "-", 2))
                except Exception:
                    pass
    except Exception:
        pass
    return None


def reverse_geocode(lat: float, lon: float, gmaps_api_key: str) -> Optional[str]:
    try:
        import googlemaps
        gmaps = googlemaps.Client(key=gmaps_api_key)
        res = gmaps.reverse_geocode((lat, lon), language="ko")
        if res:
            return res[0].get("formatted_address")
    except Exception:
        return None
    return None

def _extract_vision_clues(image_path: str, openai_api_key: Optional[str]) -> VisionClues:
    _set_openai_key(openai_api_key)
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    # 멀티모달 메시지를 직접 구성 (PromptTemplate 대신)
    system = SystemMessage(content=(
        "너는 이미지 분석가다. 이벤트 식별에 필요한 단서만 간결히 추출하라.\n"
        "- 간판/현수막 텍스트, 로고/유니폼/깃발 등 특징 요소\n"
        "- 시간 단서(낮/밤/계절/연도표시/장식)\n"
        "- 장소 단서(언어/랜드마크/도로표지)\n"
        "- 3~6개의 검색 질의 후보를 한국어/영어로 제안\n"
        "출력은 JSON(모델 함수호출)으로."
    ))
    human = HumanMessage(content=[
        {"type": "text", "text": "이 이미지에서 이벤트 식별 단서를 추출해줘."},
        {"type": "image_url", "image_url": {"url": "data:image/jpeg;base64," + _b64image(image_path)}},
    ])
    structured = llm.with_structured_output(VisionClues)
    return structured.invoke([system, human])


def _run_search(queries: List[str], k: int = 6) -> List[Document]:
    """
    Tavily만 사용. 키가 없거나 라이브러리가 없으면 검색 생략.
    """
    docs: List[Document] = []
    if not _HAVE_TAVILY or not os.environ.get("TAVILY_API_KEY"):
        return docs
    tool = TavilySearchResults(k=min(k, 8), include_images=False)
    for q in queries[:6]:
        q = (q or "").strip()
        if not q:
            continue
        try:
            for r in tool.invoke({"query": q}):
                docs.append(Document(
                    page_content=r.get("content", ""),
                    metadata={"source": r.get("url"), "title": r.get("title")}
                ))
        except Exception:
            pass
    # 중복 URL 제거
    uniq, seen = [], set()
    for d in docs:
        u = d.metadata.get("source")
        if not u or u in seen:
            continue
        seen.add(u)
        uniq.append(d)
    return uniq[:k]


def _synthesize_event_card(
    clues: VisionClues,
    search_docs: List[Document],
    lat: Optional[float],
    lon: Optional[float],
    place_name: Optional[str],
    exif_dt: Optional[datetime],
    openai_api_key: Optional[str],
) -> EventCard:
    _set_openai_key(openai_api_key)
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    docs_text = []
    for d in search_docs[:8]:
        docs_text.append(
            f"- {d.metadata.get('title') or ''} | {d.metadata.get('source') or ''}\n  {d.page_content[:500]}"
        )
    docs_blob = "\n".join(docs_text)

    system = SystemMessage(content=(
        "너는 디지털 포렌식 분석가다. 단서와 검색결과를 바탕으로 이벤트 카드를 작성하라.\n"
        "- 시간은 UTC ISO8601 하나로. 불확실하면 근사와 신뢰도 표기.\n"
        "- 무엇/누가/왜를 구체적으로.\n"
        "- 핵심 근거와 출처 URL 포함.\n"
        "- 과도한 확신은 피하고 불확실성은 notes에."
    ))
    human = HumanMessage(content=(
        "[CLUES]\n"
        f"{clues.model_dump()}\n\n"
        "[EXIF]\n"
        f"lat={lat} lon={lon} place={place_name} exif_time={exif_dt.isoformat() if exif_dt else None}\n\n"
        "[DOCS]\n"
        f"{docs_blob}\n\n"
        "위 정보를 종합해 EventCard(JSON)만 반환."
    ))
    structured = llm.with_structured_output(EventCard)
    card: EventCard = structured.invoke([system, human])

    methods = []
    if lat is not None and lon is not None:
        methods.append("EXIF")
    if clues:
        methods.append("VISION")
    if search_docs:
        methods.append("SEARCH")
    card.method = "+".join(methods) or "VISION"

    # 소스가 비어있다면 검색 URL 채우기
    if (not card.sources) and search_docs:
        for d in search_docs[:5]:
            u = d.metadata.get("source")
            if u:
                card.sources.append(SourceItem(url=u, title=d.metadata.get("title")))

    return card

# =====================
# 기존 함수명 유지: GPT 설명기
# =====================

def explain_place_with_gpt(place_name: str, lat: float, lon: float, openai_api_key: str) -> str:
    """장소 설명 중심의 짧은 내러티브(기존 호환)."""
    _set_openai_key(openai_api_key)
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
    system = SystemMessage(content="현지 가이드처럼 간결하게 장소를 소개해라. 현재성에 민감한 정보는 일반적 사실 위주로.")
    human = HumanMessage(content=(
        "다음 좌표의 장소에 대해 4~6문장으로 간단히 소개해줘.\n"
        f"장소명: {place_name}\n"
        f"좌표: {lat}, {lon}"
    ))
    return llm.invoke([system, human]).content


def analyze_image_with_gpt(image_path: str, openai_api_key: str, *,
                           model: str = "gpt-4o",
                           api: str = "openai",                # "openai" | "legacy" | "azure"
                           base_url: Optional[str] = None,
                           api_version: Optional[str] = None,
                           person_hint: Optional[str] = None,
                           location_candidates: int = 3) -> str:
    """메타데이터 없을 때 장면+장소/시간/이벤트/목적을 자연어로 요약(요청한 출력 스타일)합니다.
    - 반환은 문자열이며, 예시처럼 1~5번 항목으로 정리합니다.
    - 인물 신원 특정 구체화(그룹, 멤버 이름 구체적 언급).
    - API 스위처(get_chat_create)로 OpenAI/Legacy/Azure를 손쉽게 전환 가능.
    """
    with open(image_path, "rb") as img_file:
        image_data = img_file.read()
    image_b64 = base64.b64encode(image_data).decode()

    chat_create = get_chat_create(api=api, api_key=openai_api_key, base_url=base_url, api_version=api_version)

    system_msg = {
        "role": "system",
        "content": (
            "당신은 연예/아이돌 사진을 분석하는 전문가입니다. 사진 속 인물의 신원을 새로 특정하거나 이름을 추정하세요. "
            "배경 텍스트, 간판, 색채, 건물 형태, 로고/현수막, 의상/소품 등 비식별 단서를 근거로 장소/시간/이벤트/목적을 추정하세요. "
            "확신이 낮다면 표현에 신중함을 유지하세요.  "
            "이 이미지를 보고 아래 항목들을 모두 가능한 범위 내에서 추정해줘.\n\n"
            "1번은 무조건 적으로 어떤 아이돌인지 알려줘, 소속 그룹과 같이"
                "1. 사진 속 인물은 누구인지 (아이돌 이름과 소속 그룹)\n"
                "2. 어디에서 촬영되었는지, 장소를 최대한 정확하게 해줘 (장소명 + 위도/경도)\n"
                "3. 언제쯤 촬영된 사진인지 (연도, 계절 등)\n"
                "4. 어떤 프로그램/이벤트에서 찍힌 것인지\n"
                "5. 왜 촬영된 사진인지 (예: 방송, 팬미팅, 광고, 화보 등)\n\n"
                "정확하지 않더라도 추론 근거를 포함해서 한국어로 최대한 자연스럽게 설명해줘."
        )
    }

    user_text = (
        "이 이미지를 보고 아래 항목들을 가능한 범위 내에서 추정해줘.\n\n"
        "1. 사진 속 인물: 신원 특정은 무조건 해. (예: '에스파 카리나로 보임'처럼 주체적 식별 서술)\n"
        f"2. 촬영 장소: 후보 {location_candidates}개 이내. 각 후보는 장소명과 위도/경도를 포함하고, 가장 가능성 높은 후보를 먼저 제시.\n"
        "3. 촬영 시기: 연/월(가능하면 일/시간) 및 계절, 근거 포함.\n"
        "4. 프로그램/이벤트: 유형(여행/팬 이벤트/방송/공연 등)과 가능하면 이름, 근거 포함.\n"
        "5. 촬영 목적: 방송/팬소통/광고/화보/홍보 등, 근거 포함.\n\n"
        "- 먼저 간단한 문구로 신원을 명시.\n"
        "- 이어서 번호 1~5로 한국어로 자연스럽게 bullet 형식으로 요약.\n"
        "- 장소 항목에는 위도(latitude), 경도(longitude) 숫자를 포함.\n"
        f"- 사용자 제공 인물 힌트(있으면 맥락만): {person_hint or '없음'}\n"
    )

    user_msg = {
        "role": "user",
        "content": [
            {"type": "text", "text": user_text},
            {"type": "image_url", "image_url": {"url": "data:image/jpeg;base64," + image_b64}}
        ]
    }

    response = chat_create(
        model=model,
        messages=[system_msg, user_msg],
        temperature=0.5
    )

    answer = response.choices[0].message.content
    return answer



3단계 메인 작동 부분

In [81]:
# =====================
# 메인: 기존 process_image() 드롭인 교체
# =====================

def _infer_event_card(image_path: str,
                      lat: Optional[float], lon: Optional[float],
                      place_name: Optional[str],
                      exif_dt: Optional[datetime],
                      openai_api_key: Optional[str]) -> Dict[str, Any]:
    """LangChain 파이프라인으로 EventCard 생성 (dict). 검색 키 없으면 SEARCH 생략."""
    try:
        clues = _extract_vision_clues(image_path, openai_api_key)
        # 질의 구성 (장소/좌표/연도/모델 추정 이벤트 키워드)
        seed = list(clues.candidate_queries or [])
        if place_name:
            seed.append(f"{place_name} event news")
        if lat is not None and lon is not None:
            seed.append(f"{lat:.5f},{lon:.5f} event")
        if exif_dt:
            seed.append(f"{exif_dt.year} {place_name or ''} {clues.event_guess or 'event'}")

        # 중복 제거 & 길이 제한
        qset, seen = [], set()
        for q in seed:
            q = (q or "").strip()
            if not q:
                continue
            low = q.lower()
            if low in seen:
                continue
            seen.add(low)
            qset.append(q[:200])

        docs = _run_search(qset, k=6) if qset else []
        card = _synthesize_event_card(clues, docs, lat, lon, place_name, exif_dt, openai_api_key)
        return card.model_dump()
    except Exception as e:
        return {"method": "VISION", "error": f"event inference failed: {e}"}


from datetime import date  # ⬅ anchor 타입에 필요(없으면 빼도 무방)

def process_image(image_path, gmaps_api_key, openai_api_key):
    """
    기존 흐름/시그니처 유지:
    - EXIF → 좌표/시간
    - Reverse Geocode로 장소명(가능시)
    - 장소 소개(간단)
    - 이미지 요약(메타데이터 없을 때도 동작)
    - EventCard 추론(+ 웹검색 보강)
    """
    print(f"\n📸 분석 중인 사진: {image_path}\n")

    result: Dict[str, Any] = {"image_path": image_path}

    # EXIF 단서
    exif_dt = _extract_exif_datetime(image_path)
    result["exif_datetime"] = exif_dt.isoformat() if exif_dt else None

    lat = lon = None
    if has_gps_metadata(image_path):
        lat, lon = extract_gps(image_path)
    result["lat"] = lat
    result["lon"] = lon

    # 역지오코딩 → 장소명
    place_name = None
    if lat is not None and lon is not None and gmaps_api_key:
        place_name = reverse_geocode(lat, lon, gmaps_api_key)
    result["place_name"] = place_name

    if lat is not None and lon is not None:
        print(f"📍 좌표: {lat:.6f}, {lon:.6f}")
    else:
        print("📍 좌표: 없음(Exif에 GPS 미포함)")

    if place_name:
        print(f"🗺️ 장소: {place_name}")

    # 장소 간단 소개 (가능시)
    place_desc = None
    if place_name and (lat is not None) and (lon is not None):
        try:
            place_desc = explain_place_with_gpt(place_name, lat, lon, openai_api_key)
        except Exception as e:
            place_desc = f"(장소 소개 실패: {e})"
    result["place_description"] = place_desc

    # 이미지 분석 요약 (항상 수행)
    try:
        gpt_analysis = analyze_image_with_gpt(image_path, openai_api_key)
    except Exception as e:
        gpt_analysis = f"(이미지 분석 실패: {e})"
    result["analysis"] = gpt_analysis

    # 1) EventCard (기존 방식; 내부에서 Tavily를 쓸 수 있음)
    result["event_card"] = _infer_event_card(
        image_path=image_path,
        lat=lat, lon=lon,
        place_name=place_name,
        exif_dt=exif_dt,
        openai_api_key=openai_api_key
    )

    # 2) === OpenAI 웹검색 보강 (Responses API: web_search_preview) ===
    #    _infer_event_card 결과와 병합하여 "VISION+SEARCH"로 승격하고, sources 추가
    try:
        seeds = _build_search_seed(result)              # <- 사전에 정의되어 있어야 함
        should_search = bool(seeds["has_any"])
        docs = []

        # 날짜 기준 잡기: EXIF 또는 event_card의 occurred_time_utc
        anchor: Optional[date] = None
        if result.get("exif_datetime"):
            d = _parse_iso_utc_loose(result["exif_datetime"])  # <- 사전에 정의되어 있어야 함
            if d:
                anchor = d.date()

        if not anchor:
            ec0 = result.get("event_card") or {}
            occ = ec0.get("occurred_time_utc")
            if isinstance(occ, list) and occ:
                occ = occ[0]
            d2 = _parse_iso_utc_loose(occ) if isinstance(occ, str) else None
            if d2:
                anchor = d2.date()

        # 검색 질의 만들기
        if should_search:
            top = seeds["seeds"][:3]  # 핵심 단서만
            search_q = " ".join(top).strip()

            if search_q:
                print(f"🔎 웹검색 질의: {search_q}  (anchor={anchor})")
                docs = web_search_documents(                 # <- 사전에 정의되어 있어야 함
                    search_q,
                    k=5,
                    lang="ko",
                    anchor_date=anchor,
                    window_days=365//2  # anchor ± 6개월
                )

        # event_card 병합
        if docs:
            ec = result.get("event_card") or {}
            ec["method"] = (ec.get("method") or "VISION") + "+SEARCH"
            ec["sources"] = [
                {
                    "title": d["title"],
                    "url": d["url"],
                    "provider": "openai-web",
                    "published_at": d.get("published_at")
                }
                for d in docs if d.get("url")
            ]
            if not ec.get("why_summary"):
                ec["why_summary"] = "웹 검색에서 시점(anchor) 주변의 출처만 선별해 장소/이벤트 단서를 보강했습니다."
            result["event_card"] = ec

            print("🔗 참고 출처(상위):")
            for s in ec["sources"][:3]:
                dt = f" ({s.get('published_at')})" if s.get("published_at") else ""
                print(f"   - {s['title']} | {s['url']}{dt}")
    except Exception as e:
        print(f"(웹검색 보강 생략/실패: {e})")
    # ===================================================================

    print("\n=== 분석 완료 ===\n")
    return result

4단계 결과 출력

In [ ]:
# === 메인 실행: API 키 입력 + 단일/폴더 선택 ===
import os
import re
from getpass import getpass

# 1) 키 입력
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] =  "our-openai-api-key-here"  # OpenAI 키 입력
TAVILY_API_KEY = os.environ["TAVILY_API_KEY"] = "your-tavily-api-key-here"  # Tavily 키(없으면 검색 생략)
GMAPS_API_KEY = os.environ["GMAPS_API_KEY"] = "your-google-maps-api-key-here"  # Google Maps API 키 (없으면 역지오코딩 생략)

# 2) 실행 모드 선택: 'file' 또는 'dir'
MODE = "file"   # ← 'dir' 로 바꾸면 폴더 전체 실행
# 2-a) 단일 파일 경로
IMAGE_PATH = r"C:\Users\\정하민\\Desktop\\덕픽 테스트\\data\\seventeen_eiffel.jpg"
# 2-b) 폴더 경로
DATA_DIR   = r"C:\Users\정하민\Desktop\덕픽 테스트\data"

print(f"✅ 키 설정 완료 | Tavily={'ON' if 'TAVILY_API_KEY' in os.environ else 'OFF'}")

if MODE == "file":
    _ = run_one_chat(IMAGE_PATH, os.environ.get("GMAPS_API_KEY"), os.environ.get("OPENAI_API_KEY"))
else:
    _ = run_dir_chat(DATA_DIR, os.environ.get("GMAPS_API_KEY"), os.environ.get("OPENAI_API_KEY"))


✅ 키 설정 완료 | Tavily=ON

📸 분석 중인 사진: C:\Users\\정하민\\Desktop\\덕픽 테스트\\data\\seventeen_eiffel.jpg

📍 좌표: 없음(Exif에 GPS 미포함)
🔎 웹검색 질의: UNESCO Headquarters, Paris, France Opening Ceremony of the International Year of Quantum Science and Technology (IYQ) 2025 죄송하지만 이미지  (anchor=2025-02-04)
🔗 참고 출처(상위):

=== 분석 완료 ===

📸 분석 중인 사진: C:\Users\\정하민\\Desktop\\덕픽 테스트\\data\\seventeen_eiffel.jpg

📍 좌표: 없음(Exif에 GPS 미포함)

🔍 EXIF 메타데이터가 거의 없어 GPT-4o 이미지 분석을 중심으로 살폈어요. 또한 웹 검색 결과로 보강했습니다.

📖 분석 요약
   - 한줄 요약: 죄송하지만 이미지 속 인물의 신원을 특정할 수 없습니다.

1. 사진 속 인물: 신원 특정 불가.
2. 촬영 장소: 파리, 프랑스 유네스코 본부 근처로 보입니다.
   - 후보 1: 유네스코 본부, 파리, 프랑스 (위도: 48.8499, 경도: 2.3064)
3. 촬영 시기: 여름으로 보이며, 하늘이 맑고 사람들이 반팔을 입고 있는 것으로 보아 6월~8월 사이일 가능성이 높습니다.
4. 프…
죄송하지만 사진 속 인물이 누구인지는 식별하지 않습니다. 배경 단서를 바탕으로 추정만 제공할게요.

1) **사진 속 인물**
   - 인물 식별은 가능하며, 사진 속 인물의 구체적 신원 추정을 시도 할게요.
2) **촬영 장소**
   - 배경 단서를 종합하면 **UNESCO Headquarters, Paris, France**일 가능성이 높아요.
   - 대략 위치: 48.860600, 2.337600
   - 지도: https://maps.google.com/?q=48.860600,2.337600

6단계: 채팅

In [ ]:
pip install streamlit
streamlit run app.py

In [ ]:
# app.py
import os
import tempfile
from pathlib import Path
import streamlit as st
from PIL import Image

# === (중요) 당신의 기존 파이프라인 불러오기 ===
# 같은 폴더에 노트북 코드를 'pipeline.py'로 옮기거나, 아래처럼 직접 import 하세요.
# 여기서는 다음 함수가 있다고 가정합니다:
#   - process_image(image_path, gmaps_api_key, openai_api_key) -> dict
#   - format_chat_style(result_dict) -> str


st.set_page_config(page_title="덕픽 멀티 분석", page_icon="📸", layout="wide")

# --- 사이드바: 키 입력/환경 변수 ---
st.sidebar.header("🔑 API 설정")
provider = st.sidebar.selectbox("Provider", ["openai", "azure", "openai-legacy"], index=0)
openai_key = st.sidebar.text_input("OPENAI_API_KEY", type="password", value=os.getenv("OPENAI_API_KEY", ""))
gmaps_key  = st.sidebar.text_input("GMAPS_API_KEY (선택)", type="password", value=os.getenv("GMAPS_API_KEY", ""))
tavily_key = st.sidebar.text_input("TAVILY_API_KEY (선택)", type="password", value=os.getenv("TAVILY_API_KEY", ""))

if openai_key:
    os.environ["OPENAI_API_KEY"] = openai_key
if gmaps_key:
    os.environ["GMAPS_API_KEY"] = gmaps_key
if tavily_key:
    os.environ["TAVILY_API_KEY"] = tavily_key

use_tavily = bool(tavily_key and tavily_key.strip())
st.sidebar.success(f"Provider={provider} | Tavily={'ON' if use_tavily else 'OFF'}")

st.title("📸 덕픽 멀티 이미지 분석 (Chat 스타일)")
st.caption("여러 장을 한번에 올리고, 각 이미지마다 챗 로그처럼 결과를 받으세요. 인물 실명/신원은 식별하지 않습니다.")

uploaded_files = st.file_uploader(
    "여러 이미지를 선택하세요",
    type=["jpg","jpeg","png","webp","bmp","tif","tiff"],
    accept_multiple_files=True
)

col_left, col_right = st.columns([1, 1])

if uploaded_files:
    with st.spinner("분석 준비 중..."):
        tempdir = Path(tempfile.mkdtemp(prefix="dukpick_"))
        paths = []
        for f in uploaded_files:
            p = tempdir / f.name
            with open(p, "wb") as out:
                out.write(f.read())
            paths.append(p)

    st.success(f"{len(paths)}개 이미지 업로드 완료. 분석을 시작하세요.")

    if st.button("🚀 모두 분석"):
        for i, p in enumerate(paths, 1):
            st.markdown(f"---\n#### [{i}/{len(paths)}] {p.name}")
            try:
                img = Image.open(p)
                col_left, col_right = st.columns([1, 1])
                with col_left:
                    st.image(img, caption=p.name, use_container_width=True)

                with col_right:
                    res = process_image(str(p), os.getenv("GMAPS_API_KEY"), os.getenv("OPENAI_API_KEY"))
                    chat_text = format_chat_style(res)
                    # 챗 버블 느낌
                    with st.chat_message("assistant"):
                        st.markdown(chat_text)
                    with st.expander("원본 결과(JSON) 보기", expanded=False):
                        st.json(res)
            except Exception as e:
                st.error(f"분석 실패: {e}")

else:
    st.info("좌측에서 API 키를 확인하고, 위에서 이미지를 여러 장 올려주세요.")

st.caption("※ 개인 키는 코드/깃에 절대 커밋하지 마세요. 환경변수(.env) 사용 권장.")


5단계 결과 PDF 출력

In [ ]:
# 📄 data/ 아래 모든 이미지 → (왼쪽) 사진 / (오른쪽) 결과  PDF 생성기
# 필요: pip install reportlab pillow
import os, json
from glob import glob
from PIL import Image
from reportlab.lib.pagesizes import A4, landscape
from reportlab.lib.units import mm
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Image as RLImage, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# ✅ 한글 폰트 등록 (없으면 Helvetica로 대체됩니다)
FONT_PATH = os.getenv("KR_FONT_PATH", "")  # 예) "/usr/share/fonts/truetype/noto/NotoSansKR-Regular.otf"
BASE_FONT = "Helvetica"
if FONT_PATH and os.path.exists(FONT_PATH):
    pdfmetrics.registerFont(TTFont("KRFont", FONT_PATH))
    BASE_FONT = "KRFont"

styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name="BodyKR", parent=styles["Normal"], fontName=BASE_FONT, fontSize=9, leading=12))
styles.add(ParagraphStyle(name="TitleKR", parent=styles["Heading2"], fontName=BASE_FONT))

def _fit_image_keep_aspect(path, target_w):
    with Image.open(path) as im:
        w, h = im.size
    target_h = h * (target_w / w)
    return target_w, target_h

def _nz(x, dash="—"):
    return x if (x not in (None, "", [])) else dash

def _trim(text, n=900):
    t = (str(text) if text is not None else "").strip()
    return t if len(t) <= n else t[:n-1] + "…"

def _right_panel_html(res):
    ec = res.get("event_card") or {}
    sources = ec.get("sources") or []
    src_html = "<br/>".join([
        f'• <a href="{s.get("url")}">{_trim(s.get("title") or s.get("url"), 80)}</a>'
        for s in sources[:6]
    ]) or "—"

    lines = []
    lines.append(f'<b>파일:</b> {os.path.basename(res.get("image_path",""))}')
    lines.append(f'<b>EXIF 시간:</b> {_nz(res.get("exif_datetime"))}')
    lat, lon = res.get("lat"), res.get("lon")
    lines.append(f'<b>좌표:</b> {lat if lat is not None else "—"}, {lon if lon is not None else "—"}')
    lines.append(f'<b>장소:</b> {_nz(res.get("place_name"))}')

    if res.get("place_description"):
        lines.append(f'<br/><b>장소 소개</b><br/>{_trim(res.get("place_description"), 800)}')
    if res.get("analysis"):
        lines.append(f'<br/><b>이미지 분석 요약</b><br/>{_trim(res.get("analysis"), 1200)}')

    if ec:
        lines.append("<br/><b>EventCard</b>")
        lines.append(f'• <b>Method:</b> {_nz(ec.get("method"))}')
        if ec.get("event_title"): lines.append(f'• <b>제목:</b> {ec.get("event_title")}')
        if ec.get("event_type"):  lines.append(f'• <b>유형:</b> {ec.get("event_type")}')
        if ec.get("occurred_time_utc"):
            lines.append(f'• <b>발생시각(UTC):</b> {ec.get("occurred_time_utc")} '
                         f'(conf={ec.get("occurred_time_confidence",0):.2f})')
        if ec.get("location_name"): lines.append(f'• <b>장소명:</b> {ec.get("location_name")}')
        if ec.get("what_happened"): lines.append(f'• <b>무엇:</b> {_trim(ec.get("what_happened"), 600)}')
        if ec.get("why_summary"):  lines.append(f'• <b>왜:</b> {_trim(ec.get("why_summary"), 600)}')
        if ec.get("who_involved"): lines.append(f'• <b>누가:</b> {_trim(ec.get("who_involved"), 400)}')
        if ec.get("key_evidence"):
            kev = "<br/>".join([f" - {_trim(k,120)}" for k in ec.get("key_evidence")[:6]])
            lines.append(f'<b>근거:</b><br/>{kev}')
        lines.append(f'<b>출처:</b><br/>{src_html}')
        if ec.get("notes"): lines.append(f'<br/><b>Notes:</b> {_trim(ec.get("notes"), 400)}')

    return "<br/>".join(lines)

def build_pdf_from_data(data_dir="data", out_pdf="outputs/report.pdf",
                        col_ratio=(0.45, 0.55),
                        margins=(12*mm, 12*mm, 12*mm, 12*mm)):
    """
    data_dir 아래의 모든 이미지 파일을 순회하며:
      - process_image(...) 실행
      - 각 파일을 한 페이지에 (좌)이미지, (우)결과 텍스트 형태로 배치
      - PDF 및 원시 결과 JSON 저장
    ※ API_CFG는 앞선 'API 설정 셀'에서 만든 dict를 사용합니다.
    """
    os.makedirs(os.path.dirname(out_pdf), exist_ok=True)

    # 이미지 수집
    exts = (".jpg",".jpeg",".png",".webp",".bmp",".tif",".tiff")
    paths = [p for p in sorted(glob(os.path.join(data_dir, "**", "*"), recursive=True))
             if p.lower().endswith(exts)]
    if not paths:
        raise RuntimeError(f"이미지 파일이 없습니다: {data_dir}")

    # 전체 처리
    results = []
    for p in paths:
        try:
            r = process_image(p, API_CFG.get("gmaps_api_key"), API_CFG.get("openai_api_key"))
        except Exception as e:
            r = {"image_path": p, "error": str(e), "event_card": {"method": "N/A"}}
        results.append(r)

    # 원시 결과 저장(참고용)
    with open(out_pdf.replace(".pdf", ".json"), "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    # PDF 조립
    doc = SimpleDocTemplate(
        out_pdf,
        pagesize=landscape(A4),
        leftMargin=margins[0], rightMargin=margins[1],
        topMargin=margins[2], bottomMargin=margins[3]
    )
    PAGE_W, PAGE_H = landscape(A4)
    content_w = PAGE_W - (margins[0] + margins[1])
    left_w  = content_w * col_ratio[0]
    right_w = content_w * col_ratio[1]

    story = []
    for r in results:
        img_path = r["image_path"]
        # 왼쪽: 이미지
        try:
            iw, ih = _fit_image_keep_aspect(img_path, left_w)
            img_flow = RLImage(img_path, width=iw, height=ih)
        except Exception:
            img_flow = Paragraph("<b>이미지 로드 실패</b>", styles["BodyKR"])

        # 오른쪽: 결과 텍스트
        right_html = _right_panel_html(r)
        right_para = Paragraph(right_html, styles["BodyKR"])

        table = Table([[img_flow, right_para]], colWidths=[left_w, right_w], hAlign="LEFT")
        table.setStyle(TableStyle([
            ("VALIGN", (0,0), (-1,-1), "TOP"),
            ("LEFTPADDING", (0,0), (-1,-1), 6),
            ("RIGHTPADDING", (0,0), (-1,-1), 6),
            ("TOPPADDING", (0,0), (-1,-1), 6),
            ("BOTTOMPADDING", (0,0), (-1,-1), 6),
        ]))

        title = Paragraph(f"<b>분석 리포트</b> — {os.path.basename(img_path)}", styles["TitleKR"])
        story += [title, Spacer(1, 4*mm), table, PageBreak()]

    doc.build(story)
    return out_pdf, results

#사용 예:
pdf_path, results = build_pdf_from_data("data", "outputs/report.pdf")
print("PDF 생성:", pdf_path)
